## 知识点
#### Canny算子
#### Hough Transform(霍夫变换)
* 直线 y = mX + b
* m、b建立二维坐标
* Hough坐标系下的线表示经过XY坐标系点的线的集合
* Hough坐标系中线的交点为目标直线的m、b值
* 垂直直线有问题 转换为极坐标下曲线的焦点


#### 练习1--对摄像头视频做Canny检测
> 电脑需要连接摄像头

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
while(True):
    ret,frame = cap.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    cannyImg = cv2.Canny(gray,230,255)
    cv2.imshow('frame',cannyImg)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


#### 练习2--对读取视频

In [1]:
import cv2


cap =cv2.VideoCapture("test2.mp4")
while(cap.isOpened()):
    ret,frame = cap.read()
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


#### 练习3--识别图片中的车道

In [123]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [124]:
#Canny算子
def cannyFunc(image):
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),0)
    canny = cv2.Canny(blur,50,150)
    return canny


In [125]:
#ROI
def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([
        [(200,height),(1100,height),(500,250)]
    ])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask,polygons,255)
    masked_image = cv2.bitwise_and(image,mask)
    return masked_image

In [126]:
# 得到直线
def make_coordinates(image,line_parameters):
    slope,intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1*(3.0/5.0))
    x1 = int((y1-intercept)/slope)
    x2 = int((y2-intercept)/slope)
    return np.array([x1,y1,x2,y2])

def average_slope_intercept(image,lines):
    left_fit=[]
    right_fit=[]
    for line in lines:
        x1,y1,x2,y2 = line.reshape(4)
        parameters = np.polyfit((x1,x2),(y1,y2),1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope<0:
            left_fit.append((slope,intercept))
        else:
            right_fit.append((slope,intercept))
    left_fit_average = np.average(left_fit,axis=0)
    right_fit_average = np.average(right_fit,axis=0)
    left_line = make_coordinates(image,left_fit_average)
    right_line = make_coordinates(image,right_fit_average)
    return np.array([left_line,right_line])

In [127]:
# 显示线段
def display_lines(image,lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2 = line.reshape(4)
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
    return line_image

In [128]:
# image = cv2.imread('test_image.png')
# lane_image = np.copy(image)
# canny_image = cannyFunc(lane_image)
# cropped_image = region_of_interest(canny)
# lines = cv2.HoughLinesP(cropped_image,2,np.pi/180,100,np.array([]),minLineLength=40,maxLineGap=5)
# averaged_lines = average_slope_intercept(lane_image,lines)
# line_image = display_lines(lane_image,averaged_lines)
# combo_image = cv2.addWeighted(lane_image,0.8,line_image,1,1)

# # plt.imshow(canny)
# # plt.imshow(cropped_image)
# plt.imshow(combo_image)
# plt.show()

In [129]:
cap =cv2.VideoCapture("test2.mp4")
while(cap.isOpened()):
    ret,frame = cap.read()
    lane_image = np.copy(frame)
    canny_image = cannyFunc(lane_image)
    cropped_image = region_of_interest(canny)
    lines = cv2.HoughLinesP(cropped_image,2,np.pi/180,100,np.array([]),minLineLength=40,maxLineGap=5)
    averaged_lines = average_slope_intercept(lane_image,lines)
    line_image = display_lines(lane_image,averaged_lines)
    combo_image = cv2.addWeighted(lane_image,0.8,line_image,1,1)
    cv2.imshow('frame',combo_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

TypeError: Expected cv::UMat for argument 'src'

## 参考资料

* [OpenCV Python Tutorial - Find Lanes for Self-Driving Cars](https://youtu.be/eLTLtUVuuy4)
* [测试视频](https://github.com/rslim087a/road-video)


In [ ]:
#### Todo 
* [ ] 为什么用极坐标理解不够透彻
* [ ] 直角坐标系到极坐标的转换
* [ ] cv2.HoughLinesP参数含义
